In [2]:
from pymongo import MongoClient


client = MongoClient("localhost", 27017)

# Assign the database to a variable
db = client.NonRelProj 

# Take the collection "Reviews"
reviews = db.Reviews

# Take the collection "MetaData"
meta = db.MetaData

When searching for restaurants, people often want to see which are the ones open in that moment. This query should therefore return the restaurants close the given location that are also open: this means that the weekday and time at which the search is performed will be provided as inputs too.

In [3]:
def seconds_to_hhmm(seconds):
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    return f"{hours:02}:{minutes:02}"

# Example usage
print(seconds_to_hhmm(55800))  # Output: "15:30"
print(seconds_to_hhmm(3600))   # Output: "01:00"
print(seconds_to_hhmm(0))      # Output: "00:00"

15:30
01:00
00:00


In [4]:
def time_to_seconds(time_str):
    # If it's already an integer, return as is
    if isinstance(time_str, int):  
        return time_str
    
    # Handle unexpected types
    if not isinstance(time_str, str):  
        return None  

    # Split the time string by ":"
    time_parts = time_str.split(":")
    
    # Check if the string is correctly 
    # formatted (should have exactly two parts)
    if len(time_parts) == 2:
        try:
            # Convert hours and minutes to integers
            hours = int(time_parts[0])
            minutes = int(time_parts[1])
            
            # Calculate total seconds from midnight
            return hours * 3600 + minutes * 60
        except ValueError:
            # Handle case where hours or 
            # minutes are not integers
            return None  
    else:
        # Handle invalid time format (not "HH:MM")
        return None  
    

print(time_to_seconds("23:30"))  # Should print 84600 
print(time_to_seconds("10:45"))  # Should print 38700 
print(time_to_seconds("14:47"))  # Should print 53220

84600
38700
53220


In [5]:
# Inputs for current time and day
my_latitude = 21.924238
my_longitude = -159.510035
# Max distance from my location in meters
max_distance = 1500  
# Input weekday
current_day = 'Thursday'
# Input time  
current_time = '13:30'  

# Convert current time to seconds from midnight
current_time_seconds = time_to_seconds(current_time)

# Query to find restaurants near you
restaurant_open_rating = meta.aggregate([
    {
        "$geoNear": {
            "near": {"type": "Point", "coordinates": [
                my_longitude, my_latitude]},
            "distanceField": "distance",
            "maxDistance": max_distance,
            "spherical": True
        }
    },
    {
        "$match": {
            "hours": {
                "$elemMatch": {
                    "day": current_day,
                    "open_time": {
                        "$lte": current_time_seconds},
                    "close_time": {
                        "$gte": current_time_seconds}
                }
            }
        }
    },
    {
        "$sort": {"avg_rating": -1}
    },
    {
        "$project": {
            "_id": 1,
            "name": 1,
            "num_of_reviews": 1,
            "avg_rating": 1,
            "hours": {
                "$filter": {
                    "input": "$hours",
                    "as": "h",
                    "cond": {"$eq": ["$$h.day", current_day]}
                }
            }
        }
    }
])

# Convert to list and format output
results = list(restaurant_open_rating)
for business in results:
    if "hours" in business and business["hours"]:
        open_time_seconds = business["hours"][0]["open_time"]
        close_time_seconds = business["hours"][0]["close_time"]
        business["open_time"] = seconds_to_hhmm(
            open_time_seconds)
        business["close_time"] = seconds_to_hhmm(
            close_time_seconds)
        # Remove raw hours after processing
        del business["hours"]  
    
    print(business)


{'_id': ObjectId('67c751ad8c4ae1c3ac1bec13'), 'name': 'Uncle Bobby’s', 'avg_rating': 5, 'num_of_reviews': 14, 'open_time': '11:00', 'close_time': '16:00'}
{'_id': ObjectId('67c751ad8c4ae1c3ac1be454'), 'name': 'Kauai Poke Co.Food Truck', 'avg_rating': 4.8, 'num_of_reviews': 58, 'open_time': '11:00', 'close_time': '14:00'}
{'_id': ObjectId('67c751b08c4ae1c3ac1c22cf'), 'name': 'The Fresh Shave', 'avg_rating': 4.8, 'num_of_reviews': 254, 'open_time': '12:00', 'close_time': '16:00'}
{'_id': ObjectId('67c751b08c4ae1c3ac1c1eb5'), 'name': 'Kalaheo Cafe & Coffee Company', 'avg_rating': 4.7, 'num_of_reviews': 867, 'open_time': '07:00', 'close_time': '14:00'}
{'_id': ObjectId('67c751af8c4ae1c3ac1c1853'), 'name': 'Brick Oven Pizza', 'avg_rating': 4.3, 'num_of_reviews': 444, 'open_time': '11:00', 'close_time': '21:00'}
